In [ ]:
knitr::opts_chunk$set(echo = TRUE)
knitr::opts_chunk$set(warning = FALSE, message = FALSE) 


### R code used to generate figures for the paper: Signatures of landscape and captivity in the gut microbiota of Southern Hairy-nosed Wombats (Lasiorhinus latifrons)

## Load libraries


In [ ]:
library(phyloseq)
library(qiime2R)
library(dplyr)
library(svglite)
library(cowplot)
library(gplots)
library(ggplot2)
library(scales)
library(ggpubr)
library(tidyr)
library(knitr)
library(microbiome)
library(microbiomeutilities)
library(VennDiagram)
library(ggVennDiagram)


## Import data



In [ ]:
ps <- qza_to_phyloseq(
  features = "QIIME2_outputs/SHNW-gut-table-final.qza",
  tree = "QIIME2_outputs/sepp-tree.qza",
  taxonomy = "QIIME2_outputs/SHNW-gut-SILVA-132.qza",
  metadata = "QIIME2_outputs/SHNW_2019_Gut_Metadata.txt"
)

#PCoA matrices
pcoa_uwUniFrac_all <- read_qza("QIIME2_outputs/SHNW-gut-Core-metrics-final-filtered-table-36346/unweighted_unifrac_pcoa_results.qza")
pcoa_wUniFrac_all <- read_qza("QIIME2_outputs/SHNW-gut-Core-metrics-final-filtered-table-36346/weighted_unifrac_pcoa_results.qza")
pcoa_uwUniFrac_wild <- read_qza("QIIME2_outputs/SHNW-gut-Core-metrics-final-filtered-table-36346-WILDonly/unweighted_unifrac_pcoa_results.qza")
pcoa_wUniFrac_wild <- read_qza("QIIME2_outputs/SHNW-gut-Core-metrics-final-filtered-table-36346-WILDonly/weighted_unifrac_pcoa_results.qza")

#Alpha divresity data
faithsPD <- read.csv("QIIME2_outputs/faithsPD.tsv", sep = '\t')
ObsFeat <- read.csv("QIIME2_outputs/observed_otus.tsv", sep = '\t')

#Venn diagram data (skip second column, which is frequency of ASVs)
captive.asvs <- read.csv("QIIME2_outputs/feature-frequency-detail-captive-200.csv", header = FALSE, row.names = 1)
wild.asvs <- read.csv("QIIME2_outputs/feature-frequency-detail-wild-750.csv", header = FALSE, row.names = 1)

brookfield.asvs <- read.csv("QIIME2_outputs/feature-frequency-detail-Brookfield.csv", header = FALSE, row.names = 1)

kooloola.asvs <- read.csv("QIIME2_outputs/feature-frequency-detail-Kooloola.csv", header = FALSE, row.names = 1)

wonga.asvs <- read.csv("QIIME2_outputs/feature-frequency-detail-Wonga.csv", header = FALSE, row.names = 1)


## Alpha diversity plots (figure 2):



In [ ]:
#This sets the order in which you wish to display the samples
level_order_adiv <- c('Kooloola', 'Brookfield', 'Wonga', 'Captive')

#Plot the data!
sp_adiv_fig_faithsPD <- ggplot(faithsPD, aes(x = factor(Population, level=level_order_adiv), y=faith_pd))
sp_adiv_fig_ObsFeat <- ggplot(ObsFeat, aes(x = factor(Population, level=level_order_adiv), y=observed_otus))

#Colour vision deficiency-friendly palette
cbp2_adiv <- c("orange", "black", "red", "blue")

#Faith's PD
#add jitterm, size, and colour
sp_adiv_fig_faithsPD + 
  #Boxplot
  geom_boxplot(size=2, outlier.shape=4, outlier.size=3, outlier.stroke=3, aes(colour=Population)) +
  #Jitter, size, colour
  geom_jitter(position=position_dodge2(0.3), size=5, aes(colour=Population)) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_adiv) +
  #Tick labels
  theme(axis.text.x = element_text(face="bold", size=20), 
        axis.text.y = element_text(face="bold", size=20),
        axis.title.x = element_text(size=24, face="bold"),
        axis.title.y = element_text(size=24, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.position = "none") +
  #legend.title = element_text(size=0),
  #legend.text = element_text(size=0),
  #legend.key = element_rect(fill = "white", color = NA),
  #legend.key.size = unit(0, "line")) +
  #Axis labels
  scale_y_continuous(breaks=seq(0,400,25)) +
  labs(x = "\nPopulation") +
  labs(y = "Faith's Phylogenetic Diversity\n")

#Save image as .svg
ggsave(filename = "Fig_AlphaDiv-FaithsPD.svg", width = 20, height = 11, dpi = 300)


#Observed OTUs
#add jitterm, size, and colour
sp_adiv_fig_ObsFeat + 
  #Boxplot
  geom_boxplot(size=2, outlier.shape=4, outlier.size=3, outlier.stroke=3, aes(colour=Population)) +
  #Jitter, size, colour
  geom_jitter(position=position_dodge2(0.3), size=5, aes(colour=Population)) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_adiv) +
  #Tick labels
  theme(axis.text.x = element_text(face="bold", size=20), 
        axis.text.y = element_text(face="bold", size=20),
        axis.title.x = element_text(size=24, face="bold"),
        axis.title.y = element_text(size=24, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.position = "none") +
  #legend.title = element_text(size=0),
  #legend.text = element_text(size=0),
  #legend.key = element_rect(fill = "white", color = NA),
  #legend.key.size = unit(0, "line")) +
  #Axis labels
  scale_y_continuous(breaks=seq(0,1000,100)) +
  labs(x = "\nPopulation") +
  labs(y = "Observed features\n")

#Save image as .svg
ggsave(filename = "Fig_AlphaDiv-ObservedFeatures.svg", width = 20, height = 11, dpi = 300)


## Beta diversity PCoA plots (figures 2 and 4):



In [ ]:
#Load metadata
metadata_pcoa <- read.csv("QIIME2_outputs/SHNW_2019_Gut_Metadata.txt", sep = '\t')

#Colour palette
cbp2_beta <- c("orange", "black", "red", "blue")

#Plot PC1 vs. PC2 (uwUniFrac_all)
pcoa_uwUniFrac_all$data$Vectors %>%
  rename("X.SampleID"=SampleID) %>%
  left_join(metadata_pcoa) %>%
  
  ggplot(aes(x=PC1, y=PC2, color=Population)) +
  scale_x_reverse() +
  geom_point(size=5) +
  xlab(paste("PC1: ", round(100*pcoa_uwUniFrac_all$data$ProportionExplained[1]), "%")) +
  ylab(paste("PC2: ", round(100*pcoa_uwUniFrac_all$data$ProportionExplained[2]), "%")) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_beta) +
  
  theme(axis.text.x = element_text(face="bold", size=14), 
        axis.text.y = element_text(face="bold", size=14),
        axis.title.x = element_text(size=20, face="bold"),
        axis.title.y = element_text(size=20, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.title = element_text(size=0),
        legend.text = element_text(size=16),
        legend.key = element_rect(fill = "white", color = NA),
        legend.key.size = unit(3.5, "line"))

#Save image
ggsave(filename = "PC1-2_uwUniFrac_all.svg", width = 20, height = 11, dpi = 300)

#Plot PC1 vs. PC3 (uwUniFrac_all)
pcoa_uwUniFrac_all$data$Vectors %>%
  rename("X.SampleID"=SampleID) %>%
  left_join(metadata_pcoa) %>%
  
  ggplot(aes(x=PC1, y=PC3, color=Population)) +
  scale_x_reverse() +
  geom_point(size=5) +
  xlab(paste("PC1: ", round(100*pcoa_uwUniFrac_all$data$ProportionExplained[1]), "%")) +
  ylab(paste("PC3: ", round(100*pcoa_uwUniFrac_all$data$ProportionExplained[3]), "%")) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_beta) +
  
  theme(axis.text.x = element_text(face="bold", size=14), 
        axis.text.y = element_text(face="bold", size=14),
        axis.title.x = element_text(size=20, face="bold"),
        axis.title.y = element_text(size=20, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.title = element_text(size=0),
        legend.text = element_text(size=16),
        legend.key = element_rect(fill = "white", color = NA),
        legend.key.size = unit(3.5, "line"))

#Save image
ggsave(filename = "PC1-3_uwUniFrac_all.svg", width = 20, height = 11, dpi = 300)


#Plot PC1 vs. PC2 (wUniFrac_all)
pcoa_wUniFrac_all$data$Vectors %>%
  rename("X.SampleID"=SampleID) %>%
  left_join(metadata_pcoa) %>%
  
  ggplot(aes(x=PC1, y=PC2, color=Population)) +
  scale_x_reverse() +
  geom_point(size=5) +
  xlab(paste("PC1: ", round(100*pcoa_wUniFrac_all$data$ProportionExplained[1]), "%")) +
  ylab(paste("PC2: ", round(100*pcoa_wUniFrac_all$data$ProportionExplained[2]), "%")) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_beta) +
  
  theme(axis.text.x = element_text(face="bold", size=14), 
        axis.text.y = element_text(face="bold", size=14),
        axis.title.x = element_text(size=20, face="bold"),
        axis.title.y = element_text(size=20, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.title = element_text(size=0),
        legend.text = element_text(size=16),
        legend.key = element_rect(fill = "white", color = NA),
        legend.key.size = unit(3.5, "line"))

#Save image
ggsave(filename = "PC1-2_wUniFrac_all.svg", width = 20, height = 11, dpi = 300)

#Plot PC1 vs. PC3 (wUniFrac_all)
pcoa_wUniFrac_all$data$Vectors %>%
  rename("X.SampleID"=SampleID) %>%
  left_join(metadata_pcoa) %>%
  
  ggplot(aes(x=PC1, y=PC3, color=Population)) +
  scale_x_reverse() +
  geom_point(size=5) +
  xlab(paste("PC1: ", round(100*pcoa_wUniFrac_all$data$ProportionExplained[1]), "%")) +
  ylab(paste("PC3: ", round(100*pcoa_wUniFrac_all$data$ProportionExplained[3]), "%")) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_beta) +
  
  theme(axis.text.x = element_text(face="bold", size=14), 
        axis.text.y = element_text(face="bold", size=14),
        axis.title.x = element_text(size=20, face="bold"),
        axis.title.y = element_text(size=20, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.title = element_text(size=0),
        legend.text = element_text(size=16),
        legend.key = element_rect(fill = "white", color = NA),
        legend.key.size = unit(3.5, "line"))

#Save image
ggsave(filename = "PC1-3_wUniFrac_all.svg", width = 20, height = 11, dpi = 300)


#################
### WILD ONLY ###
#################


cbp2_beta_wild <- c("orange", "red", "blue")



#Plot PC1 vs. PC2 (uwUniFrac_wild)
pcoa_uwUniFrac_wild$data$Vectors %>%
  rename("X.SampleID"=SampleID) %>%
  left_join(metadata_pcoa) %>%
  
  ggplot(aes(x=PC1, y=PC2, color=Population)) +
  scale_x_reverse() +
  geom_point(size=5) +
  xlab(paste("PC1: ", round(100*pcoa_uwUniFrac_wild$data$ProportionExplained[1]), "%")) +
  ylab(paste("PC2: ", round(100*pcoa_uwUniFrac_wild$data$ProportionExplained[2]), "%")) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_beta_wild) +
  
  theme(axis.text.x = element_text(face="bold", size=14), 
        axis.text.y = element_text(face="bold", size=14),
        axis.title.x = element_text(size=20, face="bold"),
        axis.title.y = element_text(size=20, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.title = element_text(size=0),
        legend.text = element_text(size=16),
        legend.key = element_rect(fill = "white", color = NA),
        legend.key.size = unit(3.5, "line"))

#Save image
ggsave(filename = "PC1-2_uwUniFrac_wild.svg", width = 20, height = 11, dpi = 300)

#Plot PC1 vs. PC3 (uwUniFrac_wild)
pcoa_uwUniFrac_wild$data$Vectors %>%
  rename("X.SampleID"=SampleID) %>%
  left_join(metadata_pcoa) %>%
  
  ggplot(aes(x=PC1, y=PC3, color=Population)) +
  scale_x_reverse() +
  geom_point(size=5) +
  xlab(paste("PC1: ", round(100*pcoa_uwUniFrac_wild$data$ProportionExplained[1]), "%")) +
  ylab(paste("PC3: ", round(100*pcoa_uwUniFrac_wild$data$ProportionExplained[3]), "%")) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_beta_wild) +
  
  theme(axis.text.x = element_text(face="bold", size=14), 
        axis.text.y = element_text(face="bold", size=14),
        axis.title.x = element_text(size=20, face="bold"),
        axis.title.y = element_text(size=20, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.title = element_text(size=0),
        legend.text = element_text(size=16),
        legend.key = element_rect(fill = "white", color = NA),
        legend.key.size = unit(3.5, "line"))

#Save image
ggsave(filename = "PC1-3_uwUniFrac_wild.svg", width = 20, height = 11, dpi = 300)


#Plot PC1 vs. PC2 (wUniFrac_wild)
pcoa_wUniFrac_wild$data$Vectors %>%
  rename("X.SampleID"=SampleID) %>%
  left_join(metadata_pcoa) %>%
  
  ggplot(aes(x=PC1, y=PC2, color=Population)) +
  scale_x_reverse() +
  geom_point(size=5) +
  xlab(paste("PC1: ", round(100*pcoa_wUniFrac_wild$data$ProportionExplained[1]), "%")) +
  ylab(paste("PC2: ", round(100*pcoa_wUniFrac_wild$data$ProportionExplained[2]), "%")) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_beta_wild) +
  
  theme(axis.text.x = element_text(face="bold", size=14), 
        axis.text.y = element_text(face="bold", size=14),
        axis.title.x = element_text(size=20, face="bold"),
        axis.title.y = element_text(size=20, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.title = element_text(size=0),
        legend.text = element_text(size=16),
        legend.key = element_rect(fill = "white", color = NA),
        legend.key.size = unit(3.5, "line"))

#Save image
ggsave(filename = "PC1-2_wUniFrac_wild.svg", width = 20, height = 11, dpi = 300)

#Plot PC1 vs. PC3 (wUniFrac_wild)
pcoa_wUniFrac_wild$data$Vectors %>%
  rename("X.SampleID"=SampleID) %>%
  left_join(metadata_pcoa) %>%
  
  ggplot(aes(x=PC1, y=PC3, color=Population)) +
  scale_x_reverse() +
  geom_point(size=5) +
  xlab(paste("PC1: ", round(100*pcoa_wUniFrac_wild$data$ProportionExplained[1]), "%")) +
  ylab(paste("PC3: ", round(100*pcoa_wUniFrac_wild$data$ProportionExplained[3]), "%")) +
  #Custom manual colours
  scale_colour_manual(values=cbp2_beta_wild) +
  
  theme(axis.text.x = element_text(face="bold", size=14), 
        axis.text.y = element_text(face="bold", size=14),
        axis.title.x = element_text(size=20, face="bold"),
        axis.title.y = element_text(size=20, face="bold"),
        axis.line = element_line(colour = "black"),
        #Background panel
        panel.background = element_rect(fill = "White"),
        panel.grid.major = element_line(colour = "white"), 
        panel.grid.minor = element_line(colour = "white"),
        #Legend
        legend.title = element_text(size=0),
        legend.text = element_text(size=16),
        legend.key = element_rect(fill = "white", color = NA),
        legend.key.size = unit(3.5, "line"))

#Save image
ggsave(filename = "PC1-3_wUniFrac_wild.svg", width = 20, height = 11, dpi = 300)


## Venn diagrams (figures 3 and 5)



In [ ]:
##Captive vs. Wild
#Store only row names (ASV names) as character vectors
captive.asv.names <- row.names(captive.asvs)
wild.asv.names <- row.names(wild.asvs)

captive.vs.wild <- list(Captive = captive.asv.names, Wild = wild.asv.names)

venn.diagram(captive.vs.wild, print.mode = c("raw","percent"), fill = c("black", "white"), cat.fontface = 2, cat.pos = 180, imagetype = "png", filename = "R_figures/captive-vs-wild-venn.png")


![Captive and Wild](./R_figures/captive-vs-wild-venn.png "Title")



In [ ]:
##Wild populations
#Store only row names (ASV names) as character vectors
brookfield.asv.names <- row.names(brookfield.asvs)
kooloola.asv.names <- row.names(kooloola.asvs)
wonga.asv.names <- row.names(wonga.asvs)

wild.comparison <- list(Brookfield = brookfield.asv.names, Kooloola = kooloola.asv.names, Wonga = wonga.asv.names)

venn.diagram(wild.comparison, print.mode = c("raw","percent"), fill = c("orange", "red", "blue"), cat.fontface = 2, cat.pos = c(320, 40, 180), imagetype = "png", filename = "R_figures/wild-populations-venn.png")


![Captive and Wild](./R_figures/wild-populations-venn.png "Title")



In [ ]:
sessionInfo()

